# Voice Assistant 

This script uses the training model script, takes speech as the input then speaks out an appropriate response 

In [8]:
import random 
import json 
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

import speech_recognition as sr
import pyttsx3 as tts
import sys
import time
import playsound
import os
from gtts import gTTS


In [29]:
r = sr.Recognizer()

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')

In [33]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def record_audio():
    with sr.Microphone() as source:
        audio = r.listen(source)
        voice_data = ''
        try:
            voice_data = r.recognize_google(audio)
        except sr.UnknownValueError:
            speak('did not understand')
        except sr.RequestError:
            speak('service is down')
        return voice_data

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list


    
def speak(audio_string):
    tts = gTTS(text=audio_string, lang='en')
    r = random.randint(1, 10000000)
    audio_file = 'audio-' + str(r) + '.mp3'
    tts.save(audio_file)
    playsound.playsound(audio_file)
    os.remove(audio_file)
    

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    return result
    

def there_exists(terms):
    for term in terms:
        if term in voice_data:
            return True
    
def leave(voice_data):
    if there_exists(["exit", "quit", "goodbye"]):
        exit()

        
        
time.sleep(1)

while 1:
    voice_data = record_audio()
    message = voice_data
    print(message)
    ints = predict_class(message)
    leave(message)
    res = get_response(ints, intents)
    speak(res)
    print(res)

is anyone there
Hello, thanks for visiting
what hours are you open
We're open every day 9am-9pm
do you accept credit cards
We accept most major credit cards
exit
Hello, thanks for visiting

Hi there, how can I help?




    Error 259 for command:
        play audio-337601.mp3 wait
    The driver cannot recognize the specified command parameter.


PlaysoundException: 
    Error 259 for command:
        play audio-337601.mp3 wait
    The driver cannot recognize the specified command parameter.